In [3]:
from searchtweets import collect_results, gen_rule_payload, load_credentials, ResultStream
from text_cleaning import clean_text
import pandas as pd
import numpy as np
premium_search_args = load_credentials(filename="./search_tweets_creds.yaml",yaml_key="search_tweets_api",env_overwrite=False)

[nltk_data] Downloading package wordnet to /Users/student/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/Users/student/Desktop/School/CISC500/sourceCode/thesisEnv/lib/python3.7/site-packages/searchtweets/credentials.py:34: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  search_creds = yaml.load(f)[yaml_key]


In [12]:
username = "theJagmeetSingh"
file_path = "../../data/{}_data.csv".format(username)


In [13]:
rule = gen_rule_payload("from:{} -is:retweet lang:en".format(username),
                        from_date="2018-10-21", #UTC 2018-10-21 00:00
                        to_date="2019-10-21",#UTC 2017-10-30 00:00
                        results_per_call=500)
rs = ResultStream(rule_payload=rule,
                  max_results=10000,
                  max_pages=500,
                  **premium_search_args)
tweets = list(rs.stream())
print(len(tweets))
# tweets = collect_results(rule,max_results=500,result_stream_args=premium_search_args)

796


In [14]:
timeline_df = pd.DataFrame(tweets)
timeline_df = timeline_df[['id','created_at', 'source', 'text','extended_tweet','favorite_count', 'retweet_count']]
timeline_df["original_author"],timeline_df["type"] = username, "tweet"
print(timeline_df.shape,list(timeline_df.columns))
timeline_df.head()

(796, 9) ['id', 'created_at', 'source', 'text', 'extended_tweet', 'favorite_count', 'retweet_count', 'original_author', 'type']


,id,created_at,source,text,extended_tweet,favorite_count,retweet_count,original_author,type
0,1186063097150074886,Sun Oct 20 23:34:13 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",Justin Trudeau could have built Pharmacare.\n\...,NaN,11072,1175,theJagmeetSingh,tweet
1,1186034986136166400,Sun Oct 20 21:42:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...","Tomorrow, Canadians get to choose an NDP gover...","{'full_text': 'Tomorrow, Canadians get to choo...",3727,566,theJagmeetSingh,tweet
2,1186010435683680256,Sun Oct 20 20:04:57 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...","""there is a risk that people will go to bed on...","{'full_text': '""there is a risk that people wi...",1622,366,theJagmeetSingh,tweet
3,1185988882782801920,Sun Oct 20 18:39:18 +0000 2019,"<a href=""https://studio.twitter.com"" rel=""nofo...",I've started doing this after every rally. It'...,{'full_text': 'I've started doing this after e...,2738,437,theJagmeetSingh,tweet
4,1185943527886188546,Sun Oct 20 15:39:05 +0000 2019,"<a href=""https://studio.twitter.com"" rel=""nofo...","On this final day of campaigning, I want to ma...",{'full_text': 'On this final day of campaignin...,5135,1207,theJagmeetSingh,tweet


In [15]:
def clean_tweet(text,extended_tweet):
    if pd.isna(extended_tweet):
        return pd.Series([clean_text(text), text])
    to_dict = dict(extended_tweet)
    return pd.Series([clean_text(to_dict["full_text"]),to_dict["full_text"]])

In [16]:
timeline_df[["text","extended_tweet"]] = timeline_df.apply(lambda x: clean_tweet(x["text"],x["extended_tweet"]),axis=1)
timeline_df["clean_text"] = timeline_df["text"]
timeline_df["original_text"] = timeline_df["extended_tweet"]
timeline_df = timeline_df.drop(["text","extended_tweet"],axis=1)
timeline_df.head()

,id,created_at,source,favorite_count,retweet_count,original_author,type,clean_text,original_text
0,1186063097150074886,Sun Oct 20 23:34:13 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",11072,1175,theJagmeetSingh,tweet,justin trudeau could built pharmacare instead ...,Justin Trudeau could have built Pharmacare.\n\...
1,1186034986136166400,Sun Oct 20 21:42:30 +0000 2019,"<a href=""http://twitter.com/download/iphone"" r...",3727,566,theJagmeetSingh,tweet,tomorrow choose government ottawa courage tax ...,"Tomorrow, Canadians get to choose an NDP gover..."
2,1186010435683680256,Sun Oct 20 20:04:57 +0000 2019,"<a href=""https://mobile.twitter.com"" rel=""nofo...",1622,366,theJagmeetSingh,tweet,risk people bed election night without taken m...,"""there is a risk that people will go to bed on..."
3,1185988882782801920,Sun Oct 20 18:39:18 +0000 2019,"<a href=""https://studio.twitter.com"" rel=""nofo...",2738,437,theJagmeetSingh,tweet,started every rally become known jagmeet jump ...,I've started doing this after every rally. It'...
4,1185943527886188546,Sun Oct 20 15:39:05 +0000 2019,"<a href=""https://studio.twitter.com"" rel=""nofo...",5135,1207,theJagmeetSingh,tweet,final day campaigning want make clear liberal ...,"On this final day of campaigning, I want to ma..."


In [17]:
csvFile = open(file_path, 'w' ,encoding='utf-8')
timeline_df.to_csv(csvFile, mode='w', index=False, encoding="utf-8")